In [1]:
#Import standard packages for model training

import sklearn as SK
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import *
from skopt import BayesSearchCV
import numpy as np
import pandas as pd
import os 
import io
from tensorboard.plugins.hparams import api as hp

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "-1"

In [2]:
#Inport train, validation and test sets

training = './data/COVID_classification2/train_fold_0.csv'
validation = './data/COVID_classification2/valid_fold_0.csv'
test = './data/COVID_classification2/test_fold_0.csv'

train_dataset = pd.read_csv(training, delimiter=',', low_memory=False)
valid_dataset = pd.read_csv(validation, delimiter=',', low_memory=False)
test_dataset = pd.read_csv(test, delimiter=',', low_memory=False)

train_dataset.head()

,ID,SMILES,Outcome
0,646,CC(=O)NC(CCC(O)=O)C(=O)NCCC1=CC=C(O)C=C1,0.0
1,392,OC(C(C)N)c1ccccc1,0.0
2,394,O=C(Nc1c(C)cccc1C(=O)OC)CN(CC)CC,0.0
3,265,CC(=O)O[Hg]c1ccccc1,0.0
4,395,NCC(O)C1=CC=CC(O)=C1,0.0


In [3]:
task_start=2
task_index = 3

# load training dataset

train_dataset = pd.concat([valid_dataset, train_dataset], axis=0).reset_index(drop=True)
y_train = np.array(train_dataset.iloc[:,2:task_index].values)
print(f"loaded y_train data: {y_train.shape}")

# load test dataset

y_test = np.array(test_dataset.iloc[:,2:task_index].values)
print(f"loaded y_test data: {y_test.shape}")

loaded y_train data: (889, 1)
loaded y_test data: (98, 1)


In [4]:
# calculate ECFP (defaut) fingerprints using RDKit

from utils.fingerprints import *

train_smiles=train_dataset["SMILES"].values
test_smiles=test_dataset["SMILES"].values
X_train = assing_fp(train_smiles,FP_SIZE,RADIUS)
X_test = assing_fp(test_smiles,FP_SIZE,RADIUS)

y_train = y_train.ravel()
y_train = np.array(y_train).astype(int)
y_test = y_test.ravel()
y_test = np.array(y_test).astype(int)

X_train.shape, X_test.shape

((889, 2048), (98, 2048))

In [5]:
#Parameters for train model

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
scorer = make_scorer(mean_squared_error)

#log-uniform: understand as search over p = exp(x) by varying x

model = BayesSearchCV(RandomForestClassifier(),
    {
    'max_features': ['auto', 'sqrt'],
    'n_estimators': [2, 200],
    "max_depth": [2, 30],
    'min_samples_leaf': [1, 20], 
    'min_samples_split': [2, 20]
    },
    n_iter=50, # Number of parameter settings that are sampled
    cv=cv,
    scoring = scorer,
    refit = True, # Refit the best estimator with the entire dataset.
    random_state=42,
    n_jobs = -1
)

model.fit(X_train, y_train)

print("Best parameters: %s" % model.best_params_)

C:\Users\Predator\anaconda3\envs\ia\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Best parameters: OrderedDict([('max_depth', 17), ('max_features', 'auto'), ('min_samples_leaf', 2), ('min_samples_split', 12), ('n_estimators', 2)])


C:\Users\Predator\anaconda3\envs\ia\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


In [6]:
#Fit model using best hyperparameters

model = RandomForestClassifier(**model.best_params_)
model.fit(X_train, y_train)

C:\Users\Predator\anaconda3\envs\ia\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


RandomForestClassifier(max_depth=17, max_features='auto', min_samples_leaf=2,
                       min_samples_split=12, n_estimators=2)

In [7]:
#Statistical characteristics of model

y_pred_train = (model.predict_proba(X_train)[:,1] >= 0.15).astype(bool)
y_pred_test = (model.predict_proba(X_test)[:,1] >= 0.15).astype(bool)

confusion = confusion_matrix(y_train, y_pred_train)
    #[row, column]
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

print(("Train set results"))
print("ACC\t%.2f" % ((TN+TP)/(TN+TP+FN+FP)))
print("MCC\t%.2f" % matthews_corrcoef(y_train, y_pred_train))
print("kappa\t%.2f" % cohen_kappa_score(y_train, y_pred_train))
print("SE\t%.2f" % (TP/(TP+FN)))
print("SP\t%.2f" % (TN/(TN+FP)))
print("PPV\t%.2f" % (TP/(TP+FP)))
print("NPV\t%.2f" % (TN/(TN+FN)))
print("TPR\t%.2f" %(TP/(TP+FN)))
print("FPR\t%.2f" %(FP/(FP+TN)))
print("F1\t%.2f" % f1_score(y_train, y_pred_train))


confusion_test = confusion_matrix(y_test, y_pred_test)
    #[row, column]
TP_test = confusion[1, 1]
TN_test = confusion[0, 0]
FP_test = confusion[0, 1]
FN_test = confusion[1, 0]

print(("Test set results"))
print("ACC\t%.2f" % ((TN_test+TP_test)/(TN_test+TP_test+FN_test+FP_test)))
print("MCC\t%.2f" % matthews_corrcoef(y_test, y_pred_test))
print("kappa\t%.2f" % cohen_kappa_score(y_test, y_pred_test))
print("SE\t%.2f" % (TP_test/(TP_test+FN_test)))
print("SP\t%.2f" % (TN_test/(TN_test+FP_test)))
print("PPV\t%.2f" % (TP_test/(TP_test+FP_test)))
print("NPV\t%.2f" % (TN_test/(TN_test+FN_test)))
print("TPR\t%.2f" %(TP_test/(TP_test+FN_test)))
print("FPR\t%.2f" %(FP_test/(FP_test+TN_test)))
print("F1\t%.2f" % f1_score(y_test, y_pred_test))

Train set results
ACC	0.81
MCC	0.40
kappa	0.34
SE	0.78
SP	0.81
PPV	0.30
NPV	0.97
TPR	0.78
FPR	0.19
F1	0.43
Test set results
ACC	0.81
MCC	0.10
kappa	0.07
SE	0.78
SP	0.81
PPV	0.30
NPV	0.97
TPR	0.78
FPR	0.19
F1	0.20
